In [ ]:
%pip install xgboost
%pip install lightgbm


In [ ]:
"""
ADHD Prediction Dataset - Feature Engineering Module

This notebook performs feature engineering on the preprocessed ADHD dataset,
including correlation analysis, feature selection, and model comparison to
identify the optimal feature set and baseline model for ADHD prediction.

Author: [Your Name]
Date: [Date]
Project: ADHD Sex Prediction
Input: Preprocessed dataset from Data Preparation module
Output: Final engineered dataset and baseline model selection
"""

import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from scipy.stats import chi2_contingency
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
import lightgbm as lgb
import xgboost as xgb

# =============================================================================
# CONFIGURATION AND SETTINGS
# =============================================================================

# Set random seeds for reproducibility
RANDOM_SEEDS = [21, 42, 100, 123, 2025]
TEST_SIZE = 0.2
CORRELATION_THRESHOLD = 0.7





In [ ]:
import joblib


def load_and_inspect_data():
    # Load preprocessed data
    data_path = r"C:\Users\04ama\OneDrive\chemistry\ADHD_SEX_Prediction\notebooks\Data Preparation\balanced_train_data.csv"
    X_train = joblib.load('data/processed/X_train_balanced.pkl')
    y_train = joblib.load('data/processed/y_train_balanced.pkl')
    X_val = joblib.load('data/processed/X_val_processed.pkl')
    y_val = joblib.load('data/processed/y_val.pkl')
    X_test = joblib.load('data/processed/X_test_processed.pkl')
    y_test = joblib.load('data/processed/y_test.pkl')

    try:
        df = pd.read_csv(data_path)
        
        print(f" Samples: {df.shape[0]:,}")
        print(f" Features: {df.shape[1]:,}")
        
       
        return  df, X_train, y_train, X_val, y_val, X_test, y_test
        
    except FileNotFoundError:
        print(f"   ❌ Error: Dataset not found at {data_path}")
        raise
    except Exception as e:
        print(f"   ❌ Error loading dataset: {str(e)}")
        raise

# Load the dataset
df, X_train, y_train, X_val, y_val, X_test, y_test = load_and_inspect_data() 


In [ ]:
# =============================================================================
# FEATURE CORRELATION ANALYSIS
# =============================================================================

def analyze_feature_correlations(df, threshold=CORRELATION_THRESHOLD):
    """
    Analyze correlations among quantitative features and visualize results.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Dataset with features to analyze
    threshold : float
        Correlation threshold for identifying highly correlated features
    
    Returns:
    --------
    tuple
        (correlation_matrix, highly_correlated_pairs)
    """

    
    # Identify quantitative columns for correlation analysis
    quant_cols = joblib.load('C:\Users\04ama\OneDrive\chemistry\ADHD_SEX_Prediction\notebooks\Modelling & Evaluation\quant_cols.json')
    print(f"Analyzing correlations among {len(quant_cols)} quantitative features...")
    
    if not quant_cols:
        print(f" No quantitative features found for correlation analysis")
        return None, []
    
    # Calculate correlation matrix
    corr = df[quant_cols].corr()
    
    # Find highly correlated pairs
    corr_pairs = corr.unstack()
    high_corr = corr_pairs[(corr_pairs > threshold) & (corr_pairs < 1.0)]
    
    print(f"\n Correlation Analysis Results:")
    print(f" Features analyzed: {len(quant_cols)}")
    print(f" Correlation threshold: {threshold}")
    print(f" Highly correlated pairs found: {len(high_corr)}")
    
    if len(high_corr) > 0:
        print(f"\n Highly Correlated Feature Pairs:")
        for (feat1, feat2), corr_val in high_corr.head(10).items():
            print(f" {feat1} -> {feat2}: {corr_val:.3f}")
    else:
        print(f"No highly correlated features above threshold")
    
    # Create correlation heatmap
   
    plt.figure(figsize=(12, 8))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', square=True,
               cbar_kws={"shrink": .8})
    plt.title(f'Quantitative Features Correlation Matrix\n({len(quant_cols)} features)', 
              fontsize=14)
    plt.tight_layout()
    plt.show()
    
    return corr, high_corr

# Analyze correlations
corr_matrix, high_corr_pairs = analyze_feature_correlations(df)


In [ ]:
# =============================================================================
# REDUNDANT FEATURE REMOVAL
# =============================================================================

def remove_highly_correlated_features(df, corr_matrix, threshold=CORRELATION_THRESHOLD):
    """
    Remove highly correlated features to reduce multicollinearity.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Dataset with features
    corr_matrix : pd.DataFrame
        Correlation matrix of features
    threshold : float
        Correlation threshold for feature removal
    
    Returns:
    --------
    tuple
        (df_filtered, dropped_features) - Dataset after removal and list of dropped features
    """
   
    
    
    # Find highly correlated pairs
    corr_pairs = corr_matrix.unstack()
    high_corr = corr_pairs[(corr_pairs > threshold) & (corr_pairs < 1.0)]
    
    # Determine which features to drop
    to_drop = set()
    feature_pairs = []
    
    for (col1, col2), corr_val in high_corr.items():
        if col1 not in to_drop and col2 not in to_drop:
            feature_pairs.append((col1, col2, corr_val))
            
            # Drop feature with more missing values (or first one if equal)
            missing_col1 = df[col1].isnull().sum()
            missing_col2 = df[col2].isnull().sum()
            
            if missing_col1 > missing_col2:
                to_drop.add(col1)
                dropped_feature = col1
                kept_feature = col2
            elif missing_col2 > missing_col1:
                to_drop.add(col2)
                dropped_feature = col2
                kept_feature = col1
            else:
                # If equal missing values, drop the first one alphabetically
                if col1 < col2:
                    to_drop.add(col2)
                    dropped_feature = col2
                    kept_feature = col1
                else:
                    to_drop.add(col1)
                    dropped_feature = col1
                    kept_feature = col2
            
            print(f" Dropping {dropped_feature} (corr with {kept_feature}: {corr_val:.3f})")
    
    # Remove highly correlated features
    original_shape = df.shape
    
    if to_drop:
        print(f"\n Feature Removal Summary:")
        print(f"  Features to drop: {len(to_drop)}")
        print(f"  Dropped features: {sorted(list(to_drop))}")
        
        df = df.drop(columns=list(to_drop))
        
        print(f"\n   📈 Dataset Shape Changes:")
        print(f"      • Before: {original_shape[0]:,} rows × {original_shape[1]:,} columns")
        print(f"      • After: {df.shape[0]:,} rows × {df.shape[1]:,} columns")
        print(f"      • Features removed: {len(to_drop)}")
        print(f"      • Features retained: {df.shape[1] - original_shape[1] + len(to_drop)}")
        
    else:
        print(f"  No highly correlated features found to drop")
        df = df.copy()
    
    return df, list(to_drop)

# Remove highly correlated features
df, dropped_features = remove_highly_correlated_features(df, corr_matrix)

In [ ]:
# =============================================================================
# MODEL COMPARISON AND EVALUATION
# =============================================================================

def evaluate_models_with_multiple_seeds(X_train, X_val, y_train, y_val, seeds=RANDOM_SEEDS):
    """
    Evaluate multiple models using preprocessed train/validation splits.
    """
    print("🏆 MODEL EVALUATION WITH MULTIPLE SEEDS")
    print("=" * 50)
    
    # Define models to evaluate
    models = {
        "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),
        "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
        "SVM": SVC(random_state=42),
        "XGBoost": xgb.XGBClassifier(random_state=42, eval_metric='logloss'),
        "LightGBM": lgb.LGBMClassifier(random_state=42, verbose=-1)
    }
    
    results = {}
    
    for name, model in models.items():
        print(f"\n   🔄 Evaluating {name}:")
        
        # Train on training set, evaluate on validation set
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        
        # Calculate macro F1-score
        report = classification_report(y_val, y_pred, output_dict=True)
        macro_f1 = report['macro avg']['f1-score']
        
        results[name] = {
            'scores': [macro_f1],  # Single score since we're using fixed splits
            'mean': macro_f1,
            'std': 0.0,  # No std since single evaluation
            'min': macro_f1,
            'max': macro_f1
        }
        
        print(f"      📊 Macro F1-Score: {macro_f1:.4f}")
    
    return results

def analyze_model_results(results):
    """
    Analyze and visualize model evaluation results.
    
    Parameters:
    -----------
    results : dict
        Model evaluation results from evaluate_models_with_multiple_seeds
    """

    print(f" Model Performance")
    
    # Sort models by mean performance
    sorted_results = sorted(results.items(), key=lambda x: x[1]['mean'], reverse=True)
    
    for rank, (name, stats) in enumerate(sorted_results, 1):
        print(f"      {rank}. {name}:")
        print(f"    Mean Macro F1: {stats['mean']:.4f}")
        print(f"     Std Deviation: {stats['std']:.4f}")
        print(f"   Stability: {'High' if stats['std'] < 0.01 else 'Medium' if stats['std'] < 0.02 else 'Low'}")
    
    # Identify best model
    best_model = sorted_results[0][0]
    best_stats = sorted_results[0][1]
    
    print(f" Best Model Selection:")
    print(f"{best_model}")
    print(f"      Mean Performance: {best_stats['mean']:.4f}")
    print(f"      Stability (Std): {best_stats['std']:.4f}")
    
    # Create visualization
 
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Bar plot of mean scores with error bars
    model_names = [name for name, _ in sorted_results]
    mean_scores = [stats['mean'] for _, stats in sorted_results]
    std_scores = [stats['std'] for _, stats in sorted_results]
    
    bars = ax1.bar(model_names, mean_scores, yerr=std_scores, capsize=5, 
                   color=['gold' if i == 0 else 'lightblue' for i in range(len(model_names))])
    ax1.set_title('Model Performance Comparison\n(Mean Macro F1-Score with Standard Deviation)')
    ax1.set_ylabel('Macro F1-Score')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, mean_val, std_val in zip(bars, mean_scores, std_scores):
        height = bar.get_height()
        ax1.text(bar.get_x() + bar.get_width()/2., height + std_val + 0.001,
                f'{mean_val:.3f}', ha='center', va='bottom', fontweight='bold')
    
    # Box plot for score distributions
    score_data = [stats['scores'] for _, stats in sorted_results]
    box_plot = ax2.boxplot(score_data, labels=model_names, patch_artist=True)

    # Coloring  the best model's box gold
    for i, patch in enumerate(box_plot['boxes']):
        if i == 0:  # Best model
            patch.set_facecolor('gold')
            patch.set_alpha(0.7)
        else:
            patch.set_facecolor('lightblue')
            patch.set_alpha(0.7)
    
    ax2.set_title('Score Distribution Across Random Seeds')
    ax2.set_ylabel('Macro F1-Score')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return best_model, best_stats



# Evaluate models
evaluation_results = evaluate_models_with_multiple_seeds(X_train, X_val, y_train, y_val)

# Analyze results
best_model_name, best_model_stats = analyze_model_results(evaluation_results)

In [ ]:
# =============================================================================
# FINAL DATASET EXPORT
# =============================================================================

def export_final_dataset(df, filename='final_dataset.csv'):
    """
    Export the final engineered dataset for model training.
    
    Parameters:
    -----------
    df : pd.DataFrame
        Final processed dataset
    filename : str
        Output filename
    """
    
    try:
        # Save final dataset
        df.to_csv(filename, index=False)
        
    except Exception as e:
        print(f"  Error exporting dataset: {str(e)}")
        raise

# Export final dataset
export_final_dataset(df, 'final_dataset.csv')

